# Planejamento da Solução

**Input - Entrada** 

1. Problema de Negócio
    -- Selecionar os clientes mais valiosos para integrar um programa de fidelização
    

2. Conjunto de dados
    -- Vendas de um e-ccomerce online durante um período de 1 ano.   
    


**Output - Saída**

A indicação de pessoas para fazer parte do programa de fidelidade "INSIDERS".

Relatório com as respostas para as seguintes perguntas:
    Quem são as pessoas elegíveis para participar do programa de Insiders ?
    
    1. Quantos clientes farão parte do grupo?
    
    2. Quais as principais características desses clientes ?
    
    3. Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    
    4. Qual a expectativa de faturamento desse grupo para os próximos meses ?
    
    5. Quais as condições para uma pessoa ser elegível ao Insiders ?
    
    6. Quais as condições para uma pessoa ser removida do Insiders ?
    
    7. Qual a garantia que o programa Insiders é melhor que o restante da base ?
    
    8. Quais ações o time de marketing pode realizar para aumentar o faturamento?

 **Tasks - Processo**

1. Quem são as pessoas elegíveis para participar do programa de Insiders ?
    --O que é ser elegível. Para a empresa o que são clientes com maior valor?

        Faturamento:
            Alto Ticket MédioAlto LTVBaixa Recência ou Alta Frequência ( tempo entre as compras )

            Alto Basket Size ( quantidade média de produtos comprados )

            Baixa probabilidade de Churn
            
            Previsão alta de LTV
            
            Alta propensão de compra

        Custo:

            Baixo número de devoluções

        Experiência de compra:  

            Média alta de avaliações
            
2. Quantos clientes farão parte de cada grupo
     --Número total de clientes
         %do grupo de insiders
     
3. Quais as principais características desses clientes?
      -- Escrever caracteristicas desses clientes
              idade
              Localização
       
      -- Escrever caracteristicas do consumo
              Atributos de clusterização
              
4. Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
        --Faturamento total do ano
        --Faturamento do grupo de insiders        
        

5. Qual a expectativa de faturamento desse grupo para os próximos meses ?
        --LTV do grupo de insiders
        --Analise de cohort
   
6. Quais as condições para uma pessoa ser elegível ao Insiders ?
        -- Definir a periodicidade
        -- Pessoas com caracteristicas parecidas com os outros insders do grupo


7. Quais as condições para uma pessoa ser removida do Insiders ?
         -- Definir a periodicidade
         -- Não estar dentro das características acima

8. Qual a garantia que o programa Insiders é melhor que o restante da base ?
        -- teste A/B
        --teste A/B baysiano
        -- Teste de hipoteses
        
        
9. Quais ações o time de marketing pode realizar para aumentar o faturamento? 
        -- Desconto
        -- frete
        -- Visita a empresa
        -- brinde
    
        



# 0.0 Imports

In [ ]:
import warnings
import pandas   as pd
import numpy    as np
import seaborn  as sns
import matplotlib.pyplot as plt

from sklearn import cluster as c
from plotly import express as px
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
warnings.filterwarnings("ignore")

## 0.01 Loading Data

In [ ]:
df_raw = pd.read_csv('C:/Users/PC/SejaUmDataScientist/DS_Clusterzizacao/data/ecommerce.csv', encoding='cp1252')
df_raw.drop('Unnamed: 8', axis=1, inplace=True)

In [ ]:
df_raw.head()

# 1.0 Data Descriptions

In [ ]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [ ]:
df1.columns

In [ ]:
cols_new =['Invoice_No', 'Stock_Code', 'Description', 'Quantity', 'Invoice_Date',
       'Unit_Price', 'Customer_ID', 'Country']


cols_new = list( (map(lambda x: x.lower(), cols_new)) )


df1.columns = cols_new

## 1.2 Data Dimensions

In [ ]:
print(f'Numer of rows : {df1.shape[0]}')
print(f'Numer of columns : {df1.shape[1]}')

## 1.3 Data Types

In [ ]:
df1.dtypes

## 1.4 Check NA

In [ ]:
(df1.isna().sum()/df1.shape[0]) * 100

In [ ]:
# correlação entre os valores NA das colunsa customerid e intem_name
isin_number = sum(np.isin(df1.loc[df1['description'].isna()].index, df1.loc[df1['customer_id'].isna()].index))
print(f'Os valores de NAs na coluna item_name estão correlacionados com a coluna costumer_id: {isin_number}')

In [ ]:
print(f'Os valores NA da coluna costumer_id, representa: {round((df1.isnull().sum()["customer_id"] * 100) / df1.shape[0], 3)}%')

- Os valores NA da coluna Customer_id representam 24.92% do dataframe. 
- É um valor significativo, porém como não seria correto preencher esses valores com a media e não há mais informações, optei por excluir esses valores NA 

## 1.5 Replace NA

In [ ]:
# Drop NA's
df1 = df1.dropna(subset=['description','customer_id'])

## 1.6 Change Dtypes

In [ ]:
# invoice date
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format='%d-%b-%y' )

# customer id
df1['customer_id'] = df1['customer_id'].astype( int  )
df1.head()

## 1.7 Descriptive Statistics

# 2.0 Feature Engeneering

In [ ]:
df2 = df1.copy()

## 2.1 Feature Creation

- Nesse primeiro ciclo criarei algumas variaveis baseadas no RFM

In [ ]:
#Data Reference
df_ref = df2.drop(['invoice_no','stock_code', 'description', 
                   'quantity', 'invoice_date', 'unit_price', 'country'], axis=1).drop_duplicates(ignore_index=True)
df_ref.head()

In [ ]:
#Gross_revenue ( quantity * price)
df2['gross_revenue'] = df2['quantity'] * df2['unit_price']

#Monetary
df_monetary = df2[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()

#Merge
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left', )

#Recency - last day purchase
df_recency = df2[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()

#merge
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')

#Frequency
df_freq = df2[['customer_id', 'invoice_no']].drop_duplicates().groupby( 'customer_id' ).count().reset_index()

#Merge
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )


In [ ]:
df_ref.head()

# 3.0 Filtering Variable

In [ ]:
df3 = df_ref.copy()

# 4.0 Exploratory Data Analysis - EDA

In [ ]:
df4 = df3.copy()

# 5.0 Data Preparation

In [ ]:
df5 = df4.copy()

# 6.0 Feature Selection

In [ ]:
df6 = df5.copy()

# 7.0 Hyper Parameter Fine-Tuning

In [ ]:
X = df6.drop(columns=['customer_id'])

In [ ]:
cluster =[2, 3, 4, 5, 6]

In [ ]:
# Within-Cluster sum of Square (WSS)
kmeans = KElbowVisualizer(c.KMeans(), k=cluster, timings=False)

kmeans.fit(X)

kmeans.show()

In [ ]:
# Silhouette  Score (SS)
kmeans = KElbowVisualizer(c.KMeans(), k=cluster, metric='silhouette', timings=False)

kmeans.fit(X)

kmeans.show()

# 8.0 Model Training

## 8.1 K-Means

In [ ]:
# Model definition
k = 3
kmeans = c.KMeans( init ='random', n_clusters=k, n_init=10, max_iter=300, random_state=42 )
# Model Training
kmeans.fit(X)
# Clustering
labels = kmeans.labels_

In [ ]:
from sklearn import metrics as m

## 8.2 Cluster Validation

In [ ]:
## Within Sum of Square 
print(f'Wss Value: {kmeans.inertia_}')

## Silhouette Score  
print('SS Value:{}'.format(m.silhouette_score(X, labels, metric='euclidean')))

# 9.0 Cluster Analysis


## 9.1 Vizualization Inspection

In [ ]:
df9 = df6.copy()
df9['cluster'] = labels

df9.head()

In [ ]:
fig = px.scatter_3d(df9, x='recency_days', y='invoice_no', z='gross_revenue', color='cluster')
fig.show()

## 9.2 Cluster Profile

In [ ]:
# Number of customer
df_cluster= df9[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100*(df_cluster['customer_id'] /df_cluster['customer_id'].sum())


# AVG Gross Revenue
df_avg_gross_revenue= df9[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()

df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')
# AVG Recency days
df_avg_recency_days = df9[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')


# AVG invoice_no
df_avg_invoice_no = df9[['invoice_no', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_invoice_no, how='inner', on='cluster')

In [ ]:
df_cluster

 Cluster 01:  ( Candidato à Insider )
- Número de customers: 6 (0.14% do customers )
- Recência em média: 7 dias
- Compras em média: 89 compras
- Receita em média: $182.182,00 dólares
    

Cluster 02:
- Número de customers: 31 (0.71 do customers )
- Recência em média: 14 dias
- Compras em média: 53 compras
- Receita em média: $40.543,52 dólares
    

Cluster 03:
- Número de customers: 4.335 (99% do customers )
- Recência em média: 92 dias
- Compras em média: 5 compras
- Receita em média: $1.372,57 dólares

# 10 Deploy


In [ ]:
#